In [ ]:
from src import NeuralTester, ExperimentConfig
import torch
from src.optimizer import PymooLearner
from src.defaults.learner_configs import PYMOO_AGE_MOEA_DEFAULT_PARAMS
from src.defaults.objective_configs import TARGETED_BOUNDARY_TESTING
from models import load_stylegan
import wandb
from _enable_logging import enable_logging

## Example usage of the Neural Tester using a MOO
We use wandb to log progress in various steps!

In [ ]:
wandb.login()
#enable_logging()  # Setup logging to be able to view logging outputs in notebooks.

In [ ]:
# Define the configurations for our experiments.
metrics = TARGETED_BOUNDARY_TESTING


conf = ExperimentConfig(
    samples_per_class=10,
    generations=150,
    mix_dim_range=(0, 8),
    sut=torch.load("../models/predictors/wrn_mnist.pkl"),  # The System under test (SUT).
    generator=load_stylegan("../models/generators/sg2_mnist.pkl"),  # The generator network.
    objectives=metrics,
    classes=10,
    save_to="results_lmt_mnist_wrn_sg2_ubt",
)

In [ ]:
learner_params = PYMOO_AGE_MOEA_DEFAULT_PARAMS
learner_params["n_var"] = conf.genome_size
learner_params["num_objectives"] = len(metrics)
learner = PymooLearner(**learner_params)  # The learner for search based optimization of candidates.

In [ ]:
# Here we initialize the Tester object.
tester = NeuralTester(
    config=conf,
    learner=learner,
    device=torch.device("cuda"),
)

In [ ]:
# We start the testing procedure.
tester.test()